In [1]:
class Struct:
    """Masquerade a dictionary with a structure-like behavior."""
    """From gprof2dot.py"""

    def __init__(self, attrs = None):
        if attrs is None:
            attrs = {}
        self.__dict__['_attrs'] = attrs

    def __getattr__(self, name):
        try:
            return self._attrs[name]
        except KeyError:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        self._attrs[name] = value

    def __str__(self):
        return str(self._attrs)

    def __repr__(self):
        return repr(self._attrs)

In [2]:
gpu_d = {}; gpu = Struct(gpu_d)
gpu.dpflop = 1870
gpu.spflop = 5600
gpu.bandwidth = 480
gpu.memcap = 24

cpu_d = {}; cpu = Struct(cpu_d)
cpu.dpflop    = 720
cpu.spflop    = 1440
cpu.bandwidth = 102
cpu.memcap    = 1536

knl_d = {}; knl = Struct(knl_d)
knl.dpflop    = 3000
knl.spflop    = 6000
knl.bandwidth = 400
knl.far_bandwidth = 90
knl.memcap    = 16

def roofline(arch):
    arch.spkink = arch.spflop * 4 / arch.bandwidth
    arch.dpkink = arch.dpflop * 8 / arch.bandwidth
roofline(cpu)
roofline(gpu)
roofline(knl)

In [3]:
def sol(new, ref):
    new.sol_dpflop = new.dpflop / ref.dpflop
    new.sol_spflop = new.spflop / ref.spflop
    new.sol_bandwidth = new.bandwidth / ref.bandwidth
sol(knl, cpu)
sol(gpu, cpu)

# Performance of accelerators in theory and in practice

## Max Hutchinson, University of Chicago

## October 2nd, 2015

### src: https://github.com/maxhutch/talks

## My perspective (porting old codes)

\begin{align*}
T_{old} &: T_{new} \\
\frac{\text{Work}_{old}}{\text{Hardware}_{old} \text{Efficiency}_{old}} &: \frac{\text{Work}_{new}}{\text{Hardware}_{new} \text{Efficiency}_{new}} \\
\frac{\text{Work}_{old}}{\text{Hardware}_{old} (1)} &: \frac{\text{Work}_{new}}{\text{Hardware}_{new} \text{Efficiency}_{new}} \\
\frac{1}{\text{Hardware}_{old} (1)} &: \frac{1}{\text{Hardware}_{new} \text{Efficiency}_{new}} \\
\end{align*}


Now, we can take two perspectives:
$$ T_{new} = T_{old} \frac{\text{Hardware}_{old}}{\text{Hardware}_{new} \text{Efficiency}_{new}} $$
or
$$\text{Efficiency}_{new} = \frac{T_{old}}{T_{new}} \frac{\text{Hardware}_{new}}{\text{Hardware}_{old}} $$

## In theory:
  0. Calculate roofline kinks and {compute, bandwidth} speeds of light
  1. Identify computational motifs in PWDFT
  2. Foreach motif:
    1. Calculate arithmetic intensity
    2. Calculate share of run-time
    3. Scale share by speed-of-light given by computational intensity
  3. Assemble shares of each motif into overall projection
  


## In practice
  5. Compare implementations to speed-of-light 

### Calculate roofline kinks and {compute, bandwidth} speeds of light

|     | CPU Hardware | GPU Hardware | KNL Hardware | Divide | GPU SOL | KNL SOL |
| --- | --- | --- | --- | ---    | --- | --- |
| DP GFlops | {{cpu.dpflop}} | {{gpu.dpflop}} | {{knl.dpflop}} | | {{"{:5.2f}".format(gpu.sol_dpflop)}} | {{"{:5.2f}".format(knl.sol_dpflop)}} |
| SP GFlops | {{cpu.spflop}} | {{gpu.spflop}} | {{knl.spflop}} | | {{"{:5.2f}".format(gpu.sol_spflop)}} | {{"{:5.2f}".format(knl.sol_spflop)}} |
| Memory Bandwidth (GiB/s) | {{cpu.bandwidth}} | {{gpu.bandwidth}} | {{knl.bandwidth}} | | {{"{:5.2f}".format(gpu.sol_bandwidth)}} | {{"{:5.2f}".format(knl.sol_bandwidth)}} |
| Divide | | | |
| DP Roofline kink | {{"{:5.2f}".format(cpu.dpkink)}} | {{"{:5.2f}".format(gpu.dpkink)}} | {{"{:5.2f}".format(knl.dpkink)}} | 
| SP Roofline kink | {{"{:5.2f}".format(cpu.spkink)}} | {{"{:5.2f}".format(gpu.spkink)}} | {{"{:5.2f}".format(knl.spkink)}} |


### Identify computational motifs

 1. Compute the action of the kinetic energy operator
 2. FFT G -> R
 3. Compute the action of the local potential
 4. FFT R -> G
 5. Project {G,R} -> $\beta$
 6. Compute the action of the non-local potential (D)
 7. Comput the action of the non-trivial overlap (Q)
 8. Construct Hamiltonian and overlap matrix
 9. Diagonalize

### Local potential, kinetic energy, (most) non-local potentials

The local potential is diagonal in r-space:
$$ \left< r \left| V^{loc} \right| \phi\right> = \left< r \left| V^{loc} \right| r\right> \left<r \middle| \phi\right>, $$
the kinetic energy is diagonal in g-space:
$$ \left< g \left| T \right| \phi\right> = \left< g \left| T \right| g\right> \left<g \middle| \phi\right>, $$
and the non-local potential and overlap matrix are block diagonal in $\beta$-space:
$$ \left< \beta_{l,a} \left| V^{nl} \right| \phi \right> = \sum_{l'}^{l_{max}} \left< \beta_{l,a} \left| V^{nl} \right| \beta_{l',a} \right> \left<\beta_{l',a} \middle| \phi\right>, $$
$$ \left< \beta_{l,a} \left| S \right| \phi \right> = \sum_{l'}^{l_{max}} \left< \beta_{l,a} \left| S \right| \beta_{l',a} \right> \left<\beta_{l',a} \middle| \phi\right>, $$
where $l_{max} \sim 10$.

$$ N_{flop} = 2 (2 l_{max} - 1) N_{\phi} $$
$$ N_{mem}  = 4 N_{\phi} + N_{op} $$
Arithmetic intensity = $(l_{max} - 1/2)$, so these are all bandwidth bound.

### Matrix elements

The matrix elements of the Hamiltonian and overlap are constructed by taking inner products of the actions with the vectors:
\begin{align*}
\left< \phi_i \left| H \right| \phi_j\right> &= \sum_g \left< \phi_i \mid g \right> \left< g \mid T \mid \phi_j \right> \\
&+ \sum_r \left< \phi_i \mid r \right> \left< r \mid V^{loc} \mid \phi_j \right> \\
&+ \sum_{a,l} \left< \phi_i \mid \beta_{a,l} \right> \left< \beta_{a,l} \mid V^{loc} \mid \phi_j \right> 
\end{align*}

$$N_{flop} = N_i N_j 4 (2 N_b - 1) \qquad N_{mem} = 2 (N_i + N_j) N_b + 2 N_i N_j $$
Arithmetic intensity, to leading order, is $2 N_i$. 

### Fourier transform

Fast Fourier transformations are used when computing the action of the local potential and when accumulating the electron density:
$$ \left<r \mid \rho \mid r \right> = \sum_{i,k} f_{i,k} \left< r \mid \phi_{i,k} \right> \left<\phi_{i,k} \mid r \right> $$

FFTs are generally memory-bound, but have more complex access patterns than dense linear algebra.

### Projection

Projection can occur in g-space or r-space.  For large problems, r-space is optimal, so we'll assume it:
$$ \left< \beta_{a,l} \mid \phi_i \right> = \sum_{r'} \sum_{r} \left< \beta_l \mid r' \right> \left< r' \mid G_a \mid r \right> \left< r \mid \phi_i \right> $$
The matrix $\left<r \mid G_a \mid r' \right>$ is a *gather* of the r-space points around atomic center $a$, and very sparse.

This is definitely memory bound, probably even memory-latency bound.

In [4]:
prof_d = {}; prof = Struct(prof_d)
prof.fft = .4
prof.mat = .3
prof.pro = .15
prof.ops = .05
prof.other = .1

In [5]:
def accelerate(profile, sols):
    res = {}
    for k in profile:
        if k in sols:
            res[k] = profile[k] / sols[k]
        else:
            res[k] = profile[k]
    return res

In [6]:
gpu_sol_d = {}; gpu_sol = Struct(gpu_sol_d)
gpu_sol.fft = gpu.sol_bandwidth
gpu_sol.mat = gpu.sol_dpflop
gpu_sol.pro = gpu.sol_bandwidth
gpu_sol.ops = gpu.sol_bandwidth
gpu_sol.other = 1.

knl_sol_d = {}; knl_sol = Struct(knl_sol_d)
knl_sol.fft = knl.sol_bandwidth
knl_sol.mat = knl.sol_dpflop
knl_sol.pro = knl.sol_bandwidth
knl_sol.ops = knl.sol_bandwidth
knl_sol.other = 1.

gpu_prof_d = accelerate(prof_d, gpu_sol_d)
gpu_prof = Struct(gpu_prof_d)

knl_prof_d = accelerate(prof_d, knl_sol_d)
knl_prof = Struct(knl_prof_d)

### Assemble shares of each motif into overall projection

| Code Section | CPU Profile | GPU Theory | KNL Theory | 
| ---     | --- | --- | --- |
| FFT        | {{prof.fft}} | {{"{:5.3f}".format(gpu_prof.fft)}} | {{"{:5.3f}".format(knl_prof.fft)}} | 
| Matrices   | {{prof.mat}} | {{"{:5.3f}".format(gpu_prof.mat)}} | {{"{:5.3f}".format(knl_prof.mat)}} |
| Projection | {{prof.pro}} | {{"{:5.3f}".format(gpu_prof.pro)}} | {{"{:5.3f}".format(knl_prof.pro)}} |
| Operators  | {{prof.ops}} | {{"{:5.3f}".format(gpu_prof.ops)}} | {{"{:5.3f}".format(knl_prof.ops)}} |
| Other      | {{prof.other}} | {{"{:5.3f}".format(gpu_prof.other)}} | {{"{:5.3f}".format(knl_prof.other)}} |
| *Accel*    | 1 | {{"{:5.2f}".format(1/sum(gpu_prof_d.values()))}} | {{"{:5.2f}".format(1/sum(knl_prof_d.values()))}} |

In [7]:
gpu_sol_d = {}; gpu_sol = Struct(gpu_sol_d)
gpu_sol.fft = 4.8
gpu_sol.mat = 7.4
gpu_sol.pro = 4.8
gpu_sol.ops = 4.8
gpu_sol.other = 1.

SiHuge_d = {}; SiHuge = Struct(SiHuge_d)
SiHuge.fft = 7307.6
SiHuge.pro = 4437.5
SiHuge.mat = 34050.8
SiHuge.other = 418.2
SiHuge.ops = 853.1 + 34.7

SiHuge_gpu_real_d = {}; SiHuge_gpu_real = Struct(SiHuge_gpu_real_d)
SiHuge_gpu_real.fft = 527.3
SiHuge_gpu_real.pro = 151.1
SiHuge_gpu_real.mat = 432.4
SiHuge_gpu_real.other = 379.3 + 1.8
SiHuge_gpu_real.ops = 41.3 + 9.0

ref = 6289.6
for k in SiHuge_gpu_real_d:
    SiHuge_gpu_real_d[k] /= ref

BhR105EX_d = {}; BhR105EX = Struct(BhR105EX_d)
BhR105EX.fft = 14669.1
BhR105EX.pro = 6429.1
BhR105EX.mat = 0
BhR105EX.other = 2188.2
BhR105EX.ops = 1233.5 + 367.9 + 219.6 + 789.4

def normalize(perf):
    total = sum(perf.values())
    for k in perf:
        perf[k] = perf[k] / total
normalize(SiHuge_d)
normalize(BhR105EX_d)

SiHuge_gpu_d = accelerate(SiHuge_d, gpu_sol_d)
SiHuge_gpu = Struct(SiHuge_gpu_d)
print(1/sum(SiHuge_gpu_d.values()))
print("Real value is 4.0")

eff_d = {}
for k in SiHuge_gpu_d:
    eff_d[k] = SiHuge_gpu_d[k] / SiHuge_gpu_real_d[k]
eff = Struct(eff_d)

gpu_sol.fft *= 2
BhR105EX_gpu = accelerate(BhR105EX_d, gpu_sol_d)
print(1/sum(BhR105EX_gpu.values()))
print("Real value is 2.9")

6.155893108413263
Real value is 4.0
4.6248741973277925
Real value is 2.9


### In practice...

| Code Section | CPU Profile  | GPU Theory | GPU Actual | Efficiency |
| ---     | --- | --- | --- |
| FFT        | {{"{:5.3f}".format(SiHuge.fft)}} | {{"{:5.3f}".format(SiHuge_gpu.fft)}} | {{"{:5.3f}".format(SiHuge_gpu_real.fft)}} | {{"{:5.3f}".format(eff.fft)}} |
| Matrices   | {{"{:5.3f}".format(SiHuge.mat)}} | {{"{:5.3f}".format(SiHuge_gpu.mat)}} | {{"{:5.3f}".format(SiHuge_gpu_real.mat)}} | {{"{:5.3f}".format(eff.mat)}} |
| Projection | {{"{:5.3f}".format(SiHuge.pro)}} | {{"{:5.3f}".format(SiHuge_gpu.pro)}} | {{"{:5.3f}".format(SiHuge_gpu_real.pro)}} | {{"{:5.3f}".format(eff.pro)}} |
| Operators  | {{"{:5.3f}".format(SiHuge.ops)}} | {{"{:5.3f}".format(SiHuge_gpu.ops)}} | {{"{:5.3f}".format(SiHuge_gpu_real.ops)}} | {{"{:5.3f}".format(eff.ops)}} |
| Other      | {{"{:5.3f}".format(SiHuge.other)}} | {{"{:5.3f}".format(SiHuge_gpu.other)}} | {{"{:5.3f}".format(SiHuge_gpu.other)}} | {{"{:5.3f}".format(eff.other)}} |
| *Accel*    | 1 | {{"{:5.2f}".format(1/sum(SiHuge_gpu_d.values()))}} | {{"{:5.2f}".format(1/sum(SiHuge_gpu_real_d.values()))}} | {{"{:5.2f}".format(sum(SiHuge_gpu_d.values())/sum(SiHuge_gpu_real_d.values()))}} |

This is from a 512-atom Si simulation, the details of which will be in the upcoming "Electronic Structure Calculations on Graphical Processing Units"

In [8]:
knl_sol_far_d = {}; knl_sol_far = Struct(knl_sol_far_d)
knl_sol_far.fft = knl.sol_bandwidth * knl.far_bandwidth / knl.bandwidth
knl_sol_far.mat = knl.sol_dpflop
knl_sol_far.pro = knl.sol_bandwidth * knl.far_bandwidth / knl.bandwidth
knl_sol_far.ops = knl.sol_bandwidth * knl.far_bandwidth / knl.bandwidth
knl_sol_far.other = 1.

knl_far_prof_d = accelerate(prof_d, knl_sol_far_d)
knl_far_prof = Struct(knl_far_prof_d)
print(knl_far_prof_d)
print(1/sum(knl_far_prof_d.values()))

{'pro': 0.17, 'mat': 0.072, 'ops': 0.05666666666666667, 'fft': 0.45333333333333337, 'other': 0.1}
1.1737089201877935


### WARNING: I've assumed computations fit into MCDRAM

| Code Section | CPU Profile | GPU Theory | KNL Theory | KNL (DDR) Theory |  
| ---     | --- | --- | --- | --- |
| FFT        | {{prof.fft}} | {{"{:5.3f}".format(gpu_prof.fft)}} | {{"{:5.3f}".format(knl_prof.fft)}} | {{"{:5.3f}".format(knl_far_prof.fft)}} | 
| Matrices   | {{prof.mat}} | {{"{:5.3f}".format(gpu_prof.mat)}} | {{"{:5.3f}".format(knl_prof.mat)}} | {{"{:5.3f}".format(knl_far_prof.mat)}} | 
| Projection | {{prof.pro}} | {{"{:5.3f}".format(gpu_prof.pro)}} | {{"{:5.3f}".format(knl_prof.pro)}} | {{"{:5.3f}".format(knl_far_prof.pro)}} | 
| Operators  | {{prof.ops}} | {{"{:5.3f}".format(gpu_prof.ops)}} | {{"{:5.3f}".format(knl_prof.ops)}} | {{"{:5.3f}".format(knl_far_prof.ops)}} | 
| Other      | {{prof.other}} | {{"{:5.3f}".format(gpu_prof.other)}} | {{"{:5.3f}".format(knl_prof.other)}} | {{"{:5.3f}".format(knl_far_prof.other)}} | 
| *Accel*    | 1 | {{"{:5.3f}".format(1/sum(gpu_prof_d.values()))}} | {{"{:5.3f}".format(1/sum(knl_prof_d.values()))}} | {{"{:5.3f}".format(1/sum(knl_far_prof_d.values()))}} | 

### src: https://github.com/maxhutch/talks